# ML Baseline

## Импорты, просмотр данных

In [1]:
# Импорты
import pandas as pd
import joblib
import numpy as np
import random
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score, recall_score, f1_score, hamming_loss, roc_auc_score
from typing import Dict, List, Union
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_selection import VarianceThreshold
random.seed(42)
np.random.seed(42)
pd.options.mode.chained_assignment = None

In [2]:
df = pd.read_parquet("D:/Datasets/HSE/Project/EDA/df_l_fin.parquet")

In [3]:
df.head(2)

,author,publication_date,hubs,comments,views,url,reading_time,individ/company,bookmarks_cnt,text_length,tags_tokens,title_tokens,rating_new,text_tokens,text_pos_tags
0,complex,2009-08-03 14:34:35+00:00,GTD,67,6800,https://habr.com/ru/articles/66091/,2.0,individual,25.0,2027,['лень' 'учись' 'работать' 'самомотивация' 'мо...,['лечение' 'приступ' 'лень'],4.0,['лишать' 'девственность' 'бложик' 'происходит...,"[NOUN, VERB, NOUN, DET, NOUN, ADV, SCONJ, PRON..."
1,popotam2,2009-07-15 20:24:31+00:00,GTD,13,3100,https://habr.com/ru/articles/64586/,1.0,individual,6.0,424,['развитие' 'работоспособность' 'организация' ...,['организация' 'рабочий' 'время' 'помощь' 'цвет'],1.0,['предлагать' 'вариант' 'сделать' 'организован...,"[VERB, ADV, NUM, NOUN, VERB, PRON, ADV, ADJ, C..."


## Подготовка данных

In [4]:
# Уберем слишком короткие (неинформативные) статьи
display(df['text_length'].describe())
df = df[df['text_length']>100].copy()
df['text_length'].describe()

count    284895.000000
mean       7571.855771
std        7980.117174
min           1.000000
25%        2104.000000
50%        5514.000000
75%       10315.000000
max      197359.000000
Name: text_length, dtype: float64

count    282640.000000
mean       7631.863576
std        7983.445597
min         101.000000
25%        2168.000000
50%        5575.000000
75%       10372.000000
max      197359.000000
Name: text_length, dtype: float64

`Убрали примерно 2000 неинформативных статей`

In [5]:
# Выделение шкалы оценок на основе рейтинга статьи,
# Это будет таргет для предсказания оценки статьи
neg_d = df[df['rating_new']<0]['rating_new'].describe()
pos_d = df[df['rating_new']>0]['rating_new'].describe()

def rating_func(row):
    rate = row['rating_new']

    if pos_d['25%'] >= rate >= neg_d['75%']:
        return('neutral')
    
    elif pos_d['75%'] >= rate > pos_d['25%']:
        return('positive')
    elif rate > pos_d['75%']:
        return('very positive')
    
    elif neg_d['75%'] > rate >= neg_d['25%']:
        return('negative')
    elif neg_d['25%'] > rate:
        return('very negative')
    
df['rating_level'] = df.apply(rating_func, axis=1)

In [6]:
# Объединяем текстовые токены в единую строку, 
# Это будет наш главный признак для предсказания тем (хабов) статей
df['text_combined'] = df['text_tokens'] + ' ' + df['title_tokens'] + ' ' + df['tags_tokens']

# Сразу удалим ненужные столбцы для облегчения вычислений
df = df.drop(columns=['tags_tokens', 'title_tokens', 'text_tokens']).copy()

In [7]:
# Извлекаем уникальные метки из тегов
unique_labels = set()
df['hubs'].str.split(', ').apply(unique_labels.update)  # Собираем уникальные метки
label_to_index = {label: idx for idx, label in enumerate(sorted(unique_labels))}  # Маппинг меток в индексы

# Преобразуем строки в списки индексов
df['hubs_encoded'] = df['hubs'].apply(lambda x: [label_to_index[label] for label in x.split(', ')])

In [8]:
df.head(2)

,author,publication_date,hubs,comments,views,url,reading_time,individ/company,bookmarks_cnt,text_length,rating_new,text_pos_tags,rating_level,text_combined,hubs_encoded
0,complex,2009-08-03 14:34:35+00:00,GTD,67,6800,https://habr.com/ru/articles/66091/,2.0,individual,25.0,2027,4.0,"[NOUN, VERB, NOUN, DET, NOUN, ADV, SCONJ, PRON...",neutral,['лишать' 'девственность' 'бложик' 'происходит...,[80]
1,popotam2,2009-07-15 20:24:31+00:00,GTD,13,3100,https://habr.com/ru/articles/64586/,1.0,individual,6.0,424,1.0,"[VERB, ADV, NUM, NOUN, VERB, PRON, ADV, ADJ, C...",neutral,['предлагать' 'вариант' 'сделать' 'организован...,[80]


In [9]:
# Функция для предсказания метрик precision, recall, f1-score и hamming-loss
def calculate_metrics(
    y_test: Union[List[int], np.ndarray],
    y_pred: Union[List[int], np.ndarray],
    average: str = '',
    zero_division: int = 0) -> Dict[str, float]:
    """
    Вычисление метрик precision, recall, f1-score и hamming-loss

    Параметры:
        y_test: Истинные значения
        y_pred: Предсказанные значения
        average: Метод усреднения для метрик precision, recall и f1-score
                       Может быть 'micro', 'macro'или 'weighted'
        zero_division: Обработка деления на ноль в метриках (0 или 1).

    Возвращает:
        Dict: Словарь с метриками
    """
    precision = round(precision_score(y_test, y_pred, average=average, zero_division=zero_division), 4)
    recall = round(recall_score(y_test, y_pred, average=average, zero_division=zero_division), 4)
    f1 = round(f1_score(y_test, y_pred, average=average, zero_division=zero_division), 4)
    hamming = round(hamming_loss(y_test, y_pred), 4)

    metrics = {'Precision': precision, 'Recall': recall,
               'F1-Score': f1, 'Hamming Loss': hamming}
    return metrics

**Подсказка для начинающих (для себя)**
* Микро-метрики учитывают вклад каждого класса пропорционально его частоте
* Макро-метрики вычисляются как среднее значение по всем классам без учета их частоты
* Взвешенные метрики учитывают частоту каждого класса

## Сэмпл датасета

`На 10% данных от датасета прогоним GridSearch и найдём оптимальные параметры для решения нашей задачи`

### Подготовка данных

In [ ]:
# Сэмпл данных - 10% от всего объёма
sample_df = df.sample(frac=0.1, random_state=42)

# Преобразование hubs_ecoded в формат матрицы с уникальными метками хабов
mlb = MultiLabelBinarizer()
y_multi = mlb.fit_transform(sample_df['hubs_encoded'])

# Удаляем метки, которые встречаются в менее чем 1% случаев
selector = VarianceThreshold(threshold=0.01)
y_multi_reduced = selector.fit_transform(y_multi)

In [11]:
y_multi_reduced.shape

(28264, 56)

`Итого осталось 56 самых популярных хабов (тем)`

### Хабы

**Нашим бейзлайном будет пайплайн TfidfVectorizer + OneVsRestClassifier(LogisticRegression)**

Предсказание будем осуществлять только на основе объединенных признаков - текста статьи, её тегов и названия

`TfidfVectorizer` преобразует текст в разреженную матрицу, а `OneVsRestClassifier` для каждой метки обучает отдельную линейную модель (`LogisticRegression`/`LinearSVC`), используя матрицу TF-IDF в качестве входных признаков

Будем передавать в `GridSearchCV` различные параметры в пайплайн для нахождения наиболее оптимальных

Для поиска лучших гиперпараметров внутри сетки будем использовать метрику `F1 с микро-усреднением`, поскольку она хорошо справляется с оценкой общей точности и полноты модели в задачах multilabel классификации на большом объеме данных с дисбалансом классов (наш случай)

In [ ]:
# Подготовка данных
X = sample_df['text_combined']
y_hubs = y_multi_reduced

# Разделение на обучающую и тестовую выборки для предсказания хабов (тем) статей
X_train, X_test, y_train_h, y_test_h = train_test_split(X, y_hubs, test_size=0.25)

# Базовый пайплайн: TfidfVectorizer + OneVsRestClassifier(LogisticRegression())
# Для логистической регрессии будем использовать solver sag как наиболее оптимальный для нашей задачи
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', OneVsRestClassifier(LogisticRegression(max_iter=1000, solver='sag')))])

In [13]:
# Определение сетки параметров
params = {
     'tfidf__max_features': [2500, 5000, 10000],  # Число признаков TF-IDF
     'tfidf__ngram_range': [(1, 2)],  # Униграммы+биграммы
     'clf__estimator': [LinearSVC(), LogisticRegression(solver='sag')],  # Тип модели
     'clf__estimator__C': [0.1, 1],  # Регуляризация
}

# StratifiedKFold
cv = KFold(n_splits=3, shuffle=True, random_state=42)

# GridSearchCV
grid_search = GridSearchCV(pipeline, params, cv=cv, scoring='f1_micro', n_jobs=6, verbose=2)
grid_search.fit(X_train, y_train_h)

# Лучшая комбинация параметров
best_params_h = grid_search.best_params_
print("Лучшие параметры:", best_params_h)

# Сохранение лучшей комбинации параметров
joblib.dump(best_params_h, 'best_params_h.pkl')

Fitting 3 folds for each of 12 candidates, totalling 36 fits
Лучшие параметры: {'clf__estimator': LinearSVC(), 'clf__estimator__C': 1, 'tfidf__max_features': 10000, 'tfidf__ngram_range': (1, 2)}


['best_params_h.pkl']

In [14]:
# Оценка на тестовой выборке
y_pred_h = grid_search.best_estimator_.predict(X_test)

# Расчёт метрик для предсказания хабов (тем)
for i in ['micro', 'macro', 'weighted']:
    lr_metrics = calculate_metrics(y_pred_h, y_test_h, i)
    print(i,'-', lr_metrics)

micro - {'Precision': 0.3123, 'Recall': 0.6637, 'F1-Score': 0.4248, 'Hamming Loss': 0.0186}
macro - {'Precision': 0.2981, 'Recall': 0.6606, 'F1-Score': 0.3943, 'Hamming Loss': 0.0186}
weighted - {'Precision': 0.3654, 'Recall': 0.6637, 'F1-Score': 0.4626, 'Hamming Loss': 0.0186}


`Выводы`
* LinearSVC показала себя как лучшая модель для предсказания хабов (тем) статей
* Модель показывает хороший уровень Recall (66%) на всех видах метрик, что делает ее полезной для нашей задачи, чтобы не пропустить релевантные темы
* Разрыв между Macro (39%) и Weighted (46%) F1-Score говорит о том, что модель недостаточно хорошо справляется с редкими классами
* Низкое значение Hamming Loss (~2%) говорит о том, что модель делает небольшое количество ошибок на уровне отдельного предсказания для каждой темы

### Рейтинг

In [15]:
# Разделение на обучающую и тестовую выборки для предсказания рейтинга статьи
y_rating = sample_df['rating_level']
y_rating = y_rating.map({'very negative': -2, 'negative': -1, 'neutral': 0, 'positive': 1, 'very positive': 2})
X_train, X_test, y_train_r, y_test_r = train_test_split(X, y_rating, test_size=0.25)

In [16]:
# Смотрим на распределение классов
y_rating.value_counts()

rating_level
 1    12326
 0     8446
 2     6287
-1      835
-2      370
Name: count, dtype: int64

`Наиболее частый класс - положительная оценка, наиболее редкий - очень негативная оценка`

In [17]:
# GridSearchCV
grid_search = GridSearchCV(pipeline, params, cv=cv, scoring='f1_micro', n_jobs=6, verbose=2)
grid_search.fit(X_train, y_train_r)

# Лучшая комбинация параметров
best_params_r = grid_search.best_params_
print("Лучшие параметры:", best_params_r)

# Сохранение лучшей комбинации параметров
joblib.dump(best_params_r, 'best_params_r.pkl')

Fitting 3 folds for each of 12 candidates, totalling 36 fits
Лучшие параметры: {'clf__estimator': LogisticRegression(solver='sag'), 'clf__estimator__C': 1, 'tfidf__max_features': 10000, 'tfidf__ngram_range': (1, 2)}


['best_params_r.pkl']

In [18]:
# Оценка на тестовой выборке
y_pred_r = grid_search.best_estimator_.predict(X_test)

# Расчёт метрик для предсказания рейтинга
for i in ['micro', 'macro', 'weighted']:
    lr_metrics = calculate_metrics(y_pred_r, y_test_r, i)
    print(i,'-', lr_metrics)

micro - {'Precision': 0.4711, 'Recall': 0.4711, 'F1-Score': 0.4711, 'Hamming Loss': 0.5289}
macro - {'Precision': 0.2633, 'Recall': 0.2779, 'F1-Score': 0.2593, 'Hamming Loss': 0.5289}
weighted - {'Precision': 0.5653, 'Recall': 0.4711, 'F1-Score': 0.5004, 'Hamming Loss': 0.5289}


`Выводы`
* Logistic Regression показала себя как лучшая модель для предсказания рейтинга статей
* Модель имеет адекватную производительность на уровне микро и взвешенных метрик для более частых классов
* Низкие макро-метрики показывают, что модель плохо работает с редкими классами (очень низкие и низкие оценки)
* Hamming Loss говорит о большом количестве ошибок (53%), что снижает практическую применимость модели, потребуется её доработка

## Полный датасет

`Теперь обучим модель и посчитаем метрики для полного набора данных`

### Подготовка данных

In [19]:
# Для чистоты эксперимента уберём данные, которые были в sample_df
df = df.drop(sample_df.index).copy()

# Преобразование hubs_ecoded в формат матрицы с уникальными метками хабов
mlb = MultiLabelBinarizer()
y_multi = mlb.fit_transform(df['hubs_encoded'])

# Удаляем метки, которые встречаются в менее чем 1% случаев
selector = VarianceThreshold(threshold=0.01)
y_multi_reduced = selector.fit_transform(y_multi)

In [20]:
y_multi_reduced.shape

(254376, 58)

`Итого осталось 58 самых популярных хабов (тем)`

### Хабы

In [ ]:
# Пайплайн с лучшими параметрами для предсказания хабов (тем) статей
pipeline_h = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', OneVsRestClassifier(LinearSVC(max_iter=1000)))])

# Настройка пайплайна с лучшими параметрами
pipeline_h.set_params(**best_params_h)

# Подготовка данных
X = df['text_combined']
y_hubs = y_multi_reduced

# Разделение на обучающую и тестовую выборки для предсказания хабов (тем) статей
X_train, X_test, y_train_h, y_test_h = train_test_split(X, y_hubs, test_size=0.25)

# Обучение и предсказание по лучшему пайплайну для хабов
pipeline_h.fit(X_train, y_train_h)
y_pred_h = pipeline_h.predict(X_test)

In [23]:
# Сохранение модели для предсказания хабов (тем) статей
joblib.dump(pipeline_h, 'pipeline_h.pkl')

# Загрузка модели
#loaded_pipeline_h = joblib.load('best_pipeline_h.pkl')

# Расчёт метрик
for i in ['micro', 'macro', 'weighted']:
    lr_metrics = calculate_metrics(y_pred_h, y_test_h, i)
    print(i,'-', lr_metrics)

micro - {'Precision': 0.3828, 'Recall': 0.7205, 'F1-Score': 0.4999, 'Hamming Loss': 0.0167}
macro - {'Precision': 0.377, 'Recall': 0.7084, 'F1-Score': 0.4766, 'Hamming Loss': 0.0167}
weighted - {'Precision': 0.4405, 'Recall': 0.7205, 'F1-Score': 0.5361, 'Hamming Loss': 0.0167}


`Выводы`
* Высокий Recall (71%-72%) указывает, что модель способна эффективно обнаруживать большинство истинных тем в статьях
* Низкий Hamming Loss (~2%) говорит о том, что в среднем модель делает мало ошибок на каждый пример, но это может быть обусловлено дисбалансом классов
* Относительно низкий Precision (38%-44%) показывает, что модель склонна к ложноположительным срабатываниям
* Модель имеет потенциал, но требует доработки, особенно в части повышения точности

### Рейтинг

In [ ]:
# Пайплайн с лучшими параметрами для предсказания рейтинга статей
pipeline_r = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', OneVsRestClassifier(LogisticRegression(max_iter=1000, solver='sag')))])

# Настройка пайплайна с лучшими параметрами
pipeline_r.set_params(**best_params_r)

# Разделение на обучающую и тестовую выборки для предсказания рейтинга статьи
y_rating = df['rating_level']
y_rating = y_rating.map({'very negative': -2, 'negative': -1, 'neutral': 0, 'positive': 1, 'very positive': 2})
X_train, X_test, y_train_r, y_test_r = train_test_split(X, y_rating, test_size=0.25)

In [25]:
y_rating.value_counts()

rating_level
 1    110927
 0     75106
 2     57297
-1      7468
-2      3578
Name: count, dtype: int64

`Здесь видим похожую картину как на сэмпле данных: наиболее частый класс - положительная оценка, наиболее редкий - очень негативная оценка`

In [26]:
# Обучение и предсказание по лучшему пайплайну для рейтинга
pipeline_r.fit(X_train, y_train_r)
y_pred_r = pipeline_r.predict(X_test)

In [ ]:
# Сохранение модели для предсказания рейтинга статей
joblib.dump(pipeline_r, 'pipeline_r.pkl')

# Расчёт метрик
for i in ['micro', 'macro', 'weighted']:
    lr_metrics = calculate_metrics(y_pred_r, y_test_r, i)
    print(i,'-', lr_metrics)

micro - {'Precision': 0.496, 'Recall': 0.496, 'F1-Score': 0.496, 'Hamming Loss': 0.504}
macro - {'Precision': 0.2889, 'Recall': 0.4111, 'F1-Score': 0.288, 'Hamming Loss': 0.504}
weighted - {'Precision': 0.5604, 'Recall': 0.496, 'F1-Score': 0.5173, 'Hamming Loss': 0.504}


`Выводы`
* Метрика Hamming Loss (~50%) говорит о том, что модель правильно классифицирует половину случаев, что недостаточно для её практического применения
* Низкие макро-метрики по сравнению с взвешенными и микро-метриками указывают на то, что модель хуже справляется с редкими классами (очень низкие и низкие оценки)
* Одинаковые значения Precision, Recall и F1 (~50%) в микро-метриках указывают на сбалансированную производительность по основным классам, но ещё есть пространство для улучшений
* Текущая модель имеет ограниченную эффективность и требует доработки. Основные проблемы связаны с дисбалансом классов и неспособностью модели правильно классифицировать редкие оценки